In [ ]:
!pip install mwclient
!pip install scikit-fuzzy

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import re
import tweepy
import mwclient
import json
import unicodedata
from datetime import datetime, timedelta
from string import punctuation

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
import skfuzzy as fuzz

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\epicp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\epicp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\epicp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\epicp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
conn = sqlite3.connect('database/tweets.db')

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# get Tweets of all users with least one Tweet with at least 250 favorites
df = pd.read_sql_query("select * from tweets where id_str in (select id_str from tweets where favorite_count>=250)", conn)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Most popular users by total faves, total RTs, average faves, average RTs, number of followers

In [4]:
df.columns

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['index', 'created_at', 'id_str', 'full_text', 'user.id_str',
       'user.followers_count', 'user.screen_name', 'user.verified',
       'retweet_count', 'favorite_count', 'neg', 'neu', 'pos', 'compound'],
      dtype='object')

In [5]:
totals = df.groupby('user.screen_name').sum()
totals

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,user.followers_count,user.verified,retweet_count,favorite_count,neg,neu,pos,compound
user.screen_name,,,,,,,,,
007,279708,2394056,4,1236,16679,0.000,3.248,0.752,3.0899
01sth02,41148,4296,0,233,1224,0.000,0.928,0.072,0.4939
06fordexplorer,20632,13657,0,46,458,0.102,0.681,0.217,0.4019
079Kal,95973,1005,0,51611,315949,0.000,0.714,0.286,0.4215
0bey_UrThirst,148277,4162,0,610,2444,0.054,0.862,0.085,0.3348
...,...,...,...,...,...,...,...,...,...
zsixkillerk,157586,20168,0,252,288,0.000,0.909,0.091,0.0258
zukibites,63119,9458,0,1329,3606,0.365,4.209,0.426,0.4695
zwstein,43749,984,0,64,289,0.091,0.909,0.000,-0.4404


In [6]:
counts = df.groupby('user.screen_name').count()
counts

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,created_at,id_str,full_text,user.id_str,user.followers_count,user.verified,retweet_count,favorite_count,neg,neu,pos,compound
user.screen_name,,,,,,,,,,,,,
007,4,4,4,4,4,4,4,4,4,4,4,4,4
01sth02,1,1,1,1,1,1,1,1,1,1,1,1,1
06fordexplorer,1,1,1,1,1,1,1,1,1,1,1,1,1
079Kal,1,1,1,1,1,1,1,1,1,1,1,1,1
0bey_UrThirst,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
zsixkillerk,1,1,1,1,1,1,1,1,1,1,1,1,1
zukibites,5,5,5,5,5,5,5,5,5,5,5,5,5
zwstein,1,1,1,1,1,1,1,1,1,1,1,1,1


In [7]:
df = totals.join(counts['index'], on='user.screen_name', rsuffix='__count')

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
k = 60
def popularity(N, t):
    return N / t * k * np.log(1 + t / k)

def fave_popularity(row):
    N = row['favorite_count']
    t = row['index__count']
    return popularity(N, t)

def rt_popularity(row):
    N = row['retweet_count']
    t = row['index__count']
    return popularity(N, t)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df['fave_popularity'] = df.apply(fave_popularity, axis=1)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
df['rt_popularity'] = df.apply(rt_popularity, axis=1)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
df = df.sort_values(by='fave_popularity', ascending=False)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
top_df = df[:10000]
top_df[['index__count', 'favorite_count', 'retweet_count', 'fave_popularity', 'rt_popularity']]

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index__count,favorite_count,retweet_count,fave_popularity,rt_popularity
user.screen_name,,,,,
realDonaldTrump,74,7069849,1578257,4.605884e+06,1.028207e+06
JoeBiden,122,7727960,1363547,4.217422e+06,7.441360e+05
KamalaHarris,162,6081785,1030798,2.947037e+06,4.994914e+05
ProjectLincoln,109,3702830,1073797,2.110723e+06,6.120963e+05
BarackObama,13,1781663,293562,1.612664e+06,2.657164e+05
...,...,...,...,...,...
jarrettryon,1,1823,388,1.807975e+03,3.848021e+02
abc7kristensze,3,1852,337,1.807188e+03,3.288457e+02
IHSAA1,2,1837,321,1.807047e+03,3.157660e+02


# Automatic popular user grouping

In [13]:
with open('api_keys.json') as f:
    keys = json.load(f)

auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# get names from Tweet handles
screen_names = top_df.index.values.tolist()
users = []

for i in range(len(top_df) // 100):
    users += api.lookup_users(screen_names=screen_names[100*i : 100*(i+1)], include_entities=False)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# get name of most relevant Wikipedia article
name_dict = {user._json['screen_name'] : user._json['name'] for user in users}
site = mwclient.Site('en.wikipedia.org')
top_articles = []
for name in list(name_dict.values()):
    top = next(site.search(name, limit=1, what='nearmatch'), None)
    if top is not None:
        top_articles.append(top.get('title'))
    else:
        top_articles.append('(None)')
        
        

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# get categories associated with Wikipedia article
wiki_cat = [list(map(lambda cat: cat.base_title, site.pages[article].categories())) for article in top_articles]

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
stopword_list = stopwords.words('english')
wnl = WordNetLemmatizer()

# From https://www.kaggle.com/alvations/basic-nlp-with-nltk#Stemming-and-Lemmatization
def penn2morphy(penntag):
    # Converts Penn Treebank tags to WordNet.
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' # if mapping isn't found, fall back to Noun.

def preprocess(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Tokenization and word removal
    words = word_tokenize(text)
    words = map(lambda w: w.lower(), words)
    words = filter(lambda w: w not in stopword_list and w not in punctuation and len(w) >= 3, words)
    words = list(words)

    # POS tagging and lemmatization
    tagged_words = pos_tag(words)
    lemmatized = [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in tagged_words]
    lemmatized = list(filter(lambda w: not any(p in w for p in punctuation) and w not in stopword_list and w not in punctuation and len(w) >= 3, lemmatized))
    return lemmatized

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# get most common terms in the categories for an idea of what categories to take
stopword_list = stopwords.words('english')
term_freqs = {}
cat_words = []

for article in wiki_cat:
    cat_words.append(preprocess(' '.join(article)))
    for term in cat_words[-1]:
        if term in term_freqs:
            term_freqs[term] += 1
        else:
            term_freqs[term] = 1
                
term_freq_df = pd.DataFrame(term_freqs, index=['freq']).transpose().sort_values('freq', ascending=False)

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
curated_cats_list = 'writer, television, politician, film, journalist, activist, actor, lawyer, representative, player, singer, democrat, candidate, businesspeople, presidential, senator, republican, actress, musician, faculty, politics, republicans, commentator, channel, science, right, blogger, critic, radio, administration, talk, comedian, president, producer, family, official, trump, novelist, guitarist, attorney, governor, military, conspiracy, personality, reporter, organization, columnist, memoirists, internet, correspondent, basketball, football, theorist, academy, sportspeople, institute, screenwriter, podcasters, newspaper, songwriter, academic, socialist, footballer, scientist, cabinet, conservative, mayor, philanthropist, composer, broadcaster, philosopher, journalists, progressivism, journalism'.split(', ')
curated_cats = set(curated_cats_list)
matched_cats = [list(set(c).intersection(curated_cats)) for c in cat_words]

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
cat_df = pd.DataFrame.from_dict({'screen_name': list(name_dict.keys()), 'name': list(name_dict.values()), 'wiki_name': top_articles, 'matched_cats': matched_cats}, orient='index').transpose()
cat_df['len'] = cat_df.apply(lambda row: len(row['matched_cats']), axis=1)
cat_df = cat_df[cat_df['len'] != 0]
cat_df

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,screen_name,name,wiki_name,matched_cats,len
1,JoeBiden,Joe Biden,Joe Biden,"[lawyer, candidate, presidential, memoirists, senator, democrat, politician, writer, football, cabinet, family, player, faculty, administration, president]",15
2,KamalaHarris,Kamala Harris,Kamala Harris,"[lawyer, candidate, presidential, memoirists, senator, democrat, attorney, politician, activist, cabinet, politics, writer, family, administration, president]",15
3,ProjectLincoln,The Lincoln Project,The Lincoln Project,"[trump, presidential]",2
4,BarackObama,Barack Obama,Barack Obama,"[lawyer, candidate, presidential, right, memoirists, senator, democrat, politician, activist, writer, academic, family, official, faculty, internet, president]",16
5,BernieSanders,Bernie Sanders,Bernie Sanders,"[producer, personality, candidate, presidential, right, senator, democrat, television, politician, activist, representative, politics, writer, progressivism, socialist, faculty, mayor, president]",18
...,...,...,...,...,...
9835,BBCSheffield,BBC Radio Sheffield,BBC Radio Sheffield,[radio],1
9837,KBibbinsDomingo,Kirsten Bibbins-Domingo,Kirsten Bibbins-Domingo,"[scientist, faculty]",2
9838,_JenniferMiel,Jennifer Miel,Jennifer Miel,[businesspeople],1
9840,saikatc,Saikat Chakrabarti,Saikat Chakrabarti,"[science, socialist, activist]",3


In [23]:
# manually curate labels (select rows to remove to perform the clustering (?), retroactively assign clusters to each account)
# mismatch_indices = [24, 34, 35, 48, 51, 53, 60, 61, 84, 86, 92, 95, 97, 108, 135, 154, 156, 162, 165, 181, 183, 190, 193, 207, 218, 224, 226, 232, 235, 239, 241, 242, 245, 249, 250, 272, 281, 287, 298, 312, 314, 319, 321, 333, 340, 346, 351, 352, 361, 362, 367, 369, 371, 375, 376, 380, 387, 396, 402, 409, 417, 419, 424, 427, 431, 436, 442, 467, 471, 474, 475, 479, 482, 488, 491, 494, 495]
mismatch_indices = [] # use this if wiki what parameter is 'nearmatch'
top_20 = term_freq_df.index.tolist()[:20]
cat_words_filt_str = []
assoc_screen_names = [] # parallel array with screen names to facilitate a join with cat_df
for i in range(len(cat_words)):
    if i not in mismatch_indices and len(matched_cats[i]) != 0:
        cat_words_filt_str.append(' '.join(filter(lambda word: word not in top_20, cat_words[i])))
        assoc_screen_names.append(cat_df['screen_name'][i])

# vectorize label assignments, normalize (otherwise there might be the tendency for users with more labels to cluster together, regardless of what those labels are), and run k-means
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(cat_words_filt_str)
X = normalize(X)
terms = vectorizer.get_feature_names()
# print(terms)

# for k in range(2, 11):
#     k_means = KMeans(n_clusters=k, n_init=100).fit(X)
#     print('Score for {} clusters: {}'.format(k, k_means.score(X)))

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
k_means = KMeans(n_clusters=6, n_init=100, random_state=1746)
labels = k_means.fit_predict(X)
centers = k_means.cluster_centers_

label_df = pd.DataFrame.from_dict({'screen_name': assoc_screen_names, 'matched_cat': cat_words_filt_str, 'label': labels})

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
uncategorized_df = label_df[(label_df['label'] == 3) | (label_df['label'] == 4)]
uncategorized_1 = list(uncategorized_df['matched_cat'])
vectorizer = CountVectorizer()
X_1 = vectorizer.fit_transform(uncategorized_1)
X_1 = normalize(X_1)

k_means_1 = KMeans(n_clusters=6, n_init=100, random_state=1746)
labels_1 = k_means_1.fit_predict(X_1)

uncategorized_df['label'] = labels_1

uncategorized_df

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,screen_name,matched_cat,label
2,ProjectLincoln,2019 establishment element joe biden 2020 presidential campaign never trump movement match political action committee mdy date june 2020 webarchive template wayback,1
5,RealCandaceO,1989 births activist element activist connecticut blogger woman politics woman youtubers blogger conspiracy theorist critic islam gun right activist nationalist virgin island descent political commentator woman activist woman blogger contain video clip hcards black conservatism commentary youtubers common category conservatism criticism political correctness critic marxism female youtubers female critic feminism new york republican different theblaze daily wire rhode island mdy date october 2019 vandalism stamford connecticut,5
7,HillaryClinton,1947 births lawyer politician woman lawyer woman politician woman methodist diplomat politician woman politician woman methodist element activists new york write english contain potentially dated methodist autobiographer feminist legal memoirists dutch descent english descent descent scottish descent welsh descent political woman right activist woman academic woman activist woman diplomat woman arkansas democrat arkansas lawyer contain potentially date 2015 contain potentially date april 2015 contain potentially dated july 2015 contain video clip curlie bill clinton candidate 2008 presidential election candidate 2016 presidential election chancellor queen belfast child right activist college republicans democratic party presidential nominee democratic party senator director walmart family bill hillary clinton feature female senator female candidate president female foreign minister first lady gentlemen arkansas grammy award winner hillary clinton illinois republicans intellectual property lawyer new york democrat obama administration cabinet member official website different break reference name formatnum argument reference error park ridge illinois politician little rock arkansas recipient four freedom award recipient order cross terra mariana 1st class rodham family different secretary senators new york arkansas school law faculty english february 2019 mdy date march 2020 webarchive template wayback wellesley college westchester county new york politician bibsys bnc bne bnf cantic cinii gnd iccu isni lccn lnb musicbrainz nara ndl nkc nla nlg nli nlk nlp nlr nsk nta rero selibr sudoc trove ulan uscongress viaf worldcatid indefinitely biography vandalism woman autobiographer woman new york politics woman member cabinet new york yale law school,3
8,skaijackson,2002 births actress actresses new york city actress child actress child actress film actress television actress hcards different mdy date november 2019 lccn viaf worldcatid vandalism,4
9,gtconway3d,1963 births lawyer lawyer element filipino descent hcards criticism donald trump federalist society member harvard lawyer boston lawyer washington massachusetts republican marlborough massachusetts match mdy date november 2018 yale law school,5
...,...,...,...
2314,FirstLadyNJ,1965 births element banker investment banker jewish descent woman banker first lady gentlemen new jersey goldman sachs new jersey democrat infobox officeholder unknown parameter middletown township new jersey virginia beach virginia sky blue owner virginia,3
2315,BBCSheffield,unsourced unsourced december 2018 bbc local radio coordinate mass medium sheffield radio station establish 1967 radio station yorkshire different british english august 2012 dmy date august 2012,1
2316,KBibbinsDomingo,scientist woman scientist scientist woman scientist epidemiologist cs1 source princeton school public international affair alumni match california san francisco california san francisco faculty ibadan orcid woman epidemiologist year birth miss living,5
2317,_JenniferMiel,1983 birth hcards businesspeople florida businesspeople washington cornell georgetown chamber commerce,5


In [52]:
uncategorized_df_2 = uncategorized_df[uncategorized_df['label'].isin([1, 2, 3, 5])]
uncategorized_2 = list(uncategorized_df_2['matched_cat'])
vectorizer = CountVectorizer()
X_2 = vectorizer.fit_transform(uncategorized_2)
X_2 = normalize(X_2)

k_means_2 = KMeans(n_clusters=6, n_init=100, random_state=1746)
labels_2 = k_means_2.fit_predict(X_2)

uncategorized_df_2['label'] = labels_2

uncategorized_df_2

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,screen_name,matched_cat,label
2,ProjectLincoln,2019 establishment element joe biden 2020 presidential campaign never trump movement match political action committee mdy date june 2020 webarchive template wayback,1
5,RealCandaceO,1989 births activist element activist connecticut blogger woman politics woman youtubers blogger conspiracy theorist critic islam gun right activist nationalist virgin island descent political commentator woman activist woman blogger contain video clip hcards black conservatism commentary youtubers common category conservatism criticism political correctness critic marxism female youtubers female critic feminism new york republican different theblaze daily wire rhode island mdy date october 2019 vandalism stamford connecticut,4
7,HillaryClinton,1947 births lawyer politician woman lawyer woman politician woman methodist diplomat politician woman politician woman methodist element activists new york write english contain potentially dated methodist autobiographer feminist legal memoirists dutch descent english descent descent scottish descent welsh descent political woman right activist woman academic woman activist woman diplomat woman arkansas democrat arkansas lawyer contain potentially date 2015 contain potentially date april 2015 contain potentially dated july 2015 contain video clip curlie bill clinton candidate 2008 presidential election candidate 2016 presidential election chancellor queen belfast child right activist college republicans democratic party presidential nominee democratic party senator director walmart family bill hillary clinton feature female senator female candidate president female foreign minister first lady gentlemen arkansas grammy award winner hillary clinton illinois republicans intellectual property lawyer new york democrat obama administration cabinet member official website different break reference name formatnum argument reference error park ridge illinois politician little rock arkansas recipient four freedom award recipient order cross terra mariana 1st class rodham family different secretary senators new york arkansas school law faculty english february 2019 mdy date march 2020 webarchive template wayback wellesley college westchester county new york politician bibsys bnc bne bnf cantic cinii gnd iccu isni lccn lnb musicbrainz nara ndl nkc nla nlg nli nlk nlp nlr nsk nta rero selibr sudoc trove ulan uscongress viaf worldcatid indefinitely biography vandalism woman autobiographer woman new york politics woman member cabinet new york yale law school,3
9,gtconway3d,1963 births lawyer lawyer element filipino descent hcards criticism donald trump federalist society member harvard lawyer boston lawyer washington massachusetts republican marlborough massachusetts match mdy date november 2018 yale law school,4
13,GavinNewsom,1967 births politician activists san francisco bay area dead external fail verification businesspeople retail chief executive drink industry businesspeople investor irish descent scottish descent restaurateur salespeople television talk show host dead external october 2017 fail verification september 2019 permanently dead external businesspeople san francisco bay area cs1 maint author parameter california democrat catholics california current democratic party governor unite gavin newsom good governor california lgbt right activist lieutenant governor california mayor san francisco newsom family greenbrae california kentfield california dyslexia russian hill san francisco santa clara alumni different television personality san francisco mdy date march 2020 webarchive template wayback bibsys cinii gnd lccn ndl nli viaf worldcatid temporarily biography,2
...,...,...,...
2314,FirstLadyNJ,1965 births element banker investment banker jewish descent woman banker first lady gentlemen new jersey goldman sachs new jersey democrat infobox officeholder unknown parameter middletown township new jersey virginia beach virginia sky blue owner virginia,3
2315,BBCSheffi

In [60]:
uncategorized_df_3 = uncategorized_df_2[uncategorized_df_2['label'].isin([0, 2, 4, 5])]
uncategorized_3 = list(uncategorized_df_3['matched_cat'])
vectorizer = CountVectorizer()
X_3 = vectorizer.fit_transform(uncategorized_3)
X_3 = normalize(X_3)

k_means_3 = KMeans(n_clusters=6, n_init=100, random_state=1746)
labels_3 = k_means_3.fit_predict(X_3)

uncategorized_df_3['label'] = labels_3

uncategorized_df_3

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,screen_name,matched_cat,label
5,RealCandaceO,1989 births activist element activist connecticut blogger woman politics woman youtubers blogger conspiracy theorist critic islam gun right activist nationalist virgin island descent political commentator woman activist woman blogger contain video clip hcards black conservatism commentary youtubers common category conservatism criticism political correctness critic marxism female youtubers female critic feminism new york republican different theblaze daily wire rhode island mdy date october 2019 vandalism stamford connecticut,4
9,gtconway3d,1963 births lawyer lawyer element filipino descent hcards criticism donald trump federalist society member harvard lawyer boston lawyer washington massachusetts republican marlborough massachusetts match mdy date november 2018 yale law school,4
13,GavinNewsom,1967 births politician activists san francisco bay area dead external fail verification businesspeople retail chief executive drink industry businesspeople investor irish descent scottish descent restaurateur salespeople television talk show host dead external october 2017 fail verification september 2019 permanently dead external businesspeople san francisco bay area cs1 maint author parameter california democrat catholics california current democratic party governor unite gavin newsom good governor california lgbt right activist lieutenant governor california mayor san francisco newsom family greenbrae california kentfield california dyslexia russian hill san francisco santa clara alumni different television personality san francisco mdy date march 2020 webarchive template wayback bibsys cinii gnd lccn ndl nli viaf worldcatid temporarily biography,3
14,Mikel_Jollett,1974 births indie rock musician hcards musician los angeles stanford gnd viaf worldcatid,2
18,sarahcpr,1977 births comedian blogger comedian woman write english tiktokers youtubers sketch comedian woman blogger woman comedian jamaican descent comedian maryland comedy youtubers commentary youtubers female tiktokers female youtubers georgia tech jamaican emigrant embed infobox templates title parameter rockville maryland different maryland college park english july 2020 mdy date november 2020 webarchive template wayback cantic gnd lccn viaf worldcatid maryland new york city san francisco,4
...,...,...,...
2312,AndrewYNg,1976 birth fail verification computer businesspeople computer scientist education businesspeople hong kong descent roboticists scientist chinese descent technology chief executive technology company founder technology venture capitalist contain text contain traditional text fail verification may 2020 hcards artificial intelligence researcher british emigrant british hong kong descent businesspeople california businesspeople london carnegie mellon alumni common category computer vision researcher machine learn researcher massachusetts institute technology los altos california raffle junior college scientist california scientist london match stanford department computer science faculty stanford school engineering faculty berkeley college engineering mdy date april 2019 dblp isni lccn mgp viaf worldcatid california london,2
2315,BBCSheffield,unsourced unsourced december 2018 bbc local radio coordinate mass medium sheffield radio station establish 1967 radio station yorkshire different british english august 2012 dmy date august 2012,0
2316,KBibbinsDomingo,scientist woman scientist scientist woman scientist epidemiologist cs1 source princeton school public international affair alumni match california san francisco california san francisco faculty ibadan orcid woman epidemiologist year birth miss living,5
2317,_JenniferMiel,1983 birth hcards businesspeople florida businesspeople washington cornell georgetown chamber commerce,4


In [84]:
uncategorized_3 = [' '.join(list(filter(lambda word: word not in ['lack', 'lacks', 'source', 'sources', 'year', 'birth', 'miss', 'living', 'member', 'bnf', 'gnd', 'isni', 'lccn', 'lnb', 'nkc', 'nta', 'sudoc', 'viaf', 'worldcatid', 'template', 'wayback'], x.split(' ')))) for x in uncategorized_3]
uncategorized_3

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['1989 births activist element activist connecticut blogger woman politics woman youtubers blogger conspiracy theorist critic islam gun right activist nationalist virgin island descent political commentator woman activist woman blogger contain video clip hcards black conservatism commentary youtubers common category conservatism criticism political correctness critic marxism female youtubers female critic feminism new york republican different theblaze daily wire rhode island mdy date october 2019 vandalism stamford connecticut',
 '1963 births lawyer lawyer element filipino descent hcards criticism donald trump federalist society harvard lawyer boston lawyer washington massachusetts republican marlborough massachusetts match mdy date november 2018 yale law school',
 '1967 births politician activists san francisco bay area dead external fail verification businesspeople retail chief executive drink industry businesspeople investor irish descent scottish descent restaurateur salespeople t

In [86]:
vectorizer = CountVectorizer()
X_3 = vectorizer.fit_transform(uncategorized_3)
X_3 = normalize(X_3)

k_means_3 = KMeans(n_clusters=6, n_init=100, random_state=1746)
labels_3 = k_means_3.fit_predict(X_3)

uncategorized_df_3['matched_cat'] = uncategorized_3
uncategorized_df_3['label'] = labels_3

uncategorized_df_3

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

,screen_name,matched_cat,label,matched_cats
5,RealCandaceO,1989 births activist element activist connecticut blogger woman politics woman youtubers blogger conspiracy theorist critic islam gun right activist nationalist virgin island descent political commentator woman activist woman blogger contain video clip hcards black conservatism commentary youtubers common category conservatism criticism political correctness critic marxism female youtubers female critic feminism new york republican different theblaze daily wire rhode island mdy date october 2019 vandalism stamford connecticut,4,1989 births activist element activist connecticut blogger woman politics woman youtubers blogger conspiracy theorist critic islam gun right activist nationalist virgin island descent political commentator woman activist woman blogger contain video clip hcards black conservatism commentary youtubers common category conservatism criticism political correctness critic marxism female youtubers female critic feminism new york republican different theblaze daily wire rhode island mdy date october 2019 vandalism stamford connecticut
9,gtconway3d,1963 births lawyer lawyer element filipino descent hcards criticism donald trump federalist society harvard lawyer boston lawyer washington massachusetts republican marlborough massachusetts match mdy date november 2018 yale law school,1,1963 births lawyer lawyer element filipino descent hcards criticism donald trump federalist society harvard lawyer boston lawyer washington massachusetts republican marlborough massachusetts match mdy date november 2018 yale law school
13,GavinNewsom,1967 births politician activists san francisco bay area dead external fail verification businesspeople retail chief executive drink industry businesspeople investor irish descent scottish descent restaurateur salespeople television talk show host dead external october 2017 fail verification september 2019 permanently dead external businesspeople san francisco bay area cs1 maint author parameter california democrat catholics california current democratic party governor unite gavin newsom good governor california lgbt right activist lieutenant governor california mayor san francisco newsom family greenbrae california kentfield california dyslexia russian hill san francisco santa clara alumni different television personality san francisco mdy date march 2020 webarchive bibsys cinii ndl nli temporarily biography,2,1967 births politician activists san francisco bay area dead external fail verification businesspeople retail chief executive drink industry businesspeople investor irish descent scottish descent restaurateur salespeople television talk show host dead external october 2017 fail verification september 2019 permanently dead external businesspeople san francisco bay area cs1 maint author parameter california democrat catholics california current democratic party governor unite gavin newsom good governor california lgbt right activist lieutenant governor california mayor san francisco newsom family greenbrae california kentfield california dyslexia russian hill san francisco santa clara alumni different television personality san francisco mdy date march 2020 webarchive bibsys cinii ndl nli temporarily biography
14,Mikel_Jollett,1974 births indie rock musician hcards musician los angeles stanford,3,1974 births indie rock musician hcards musician los angeles stanford
18,sarahcpr,1977 births comedian blogger comedian woman write english tiktokers youtubers sketch comedian woman blogger woman comedian jamaican descent comedian maryland comedy youtubers commentary youtubers female tiktokers female youtubers georgia tech jamaican emigrant embed infobox templates title parameter rockville maryland different maryland college park english july 2020 mdy date november 2020 webarchive cantic maryland new york city san francisco,4,1977 births comedian blogger comedian woman write english tiktokers youtubers sketch comedian woman blogger woma

In [95]:
temp = uncategorized_df_3[uncategorized_df_3['label'] == 5]
print(len(temp))
temp

135


C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,screen_name,matched_cat,label
100,mehdirhasan,1979 births muslim british journalist british muslim jazeera alumni christ church oxford hcards british shia muslim british expatriate unite british journalist british british british indian descent cs1 cs1 maint archive copy title common category engvarb august 2014 huffpost columnists citation bare urls citation title educate merchant taylor school northwood dmy date august 2014 webarchive archiveis webarchive nli indefinitely biography,5
118,morethanmySLE,1929 births 2013 death need additional reference alumni cambridge need additional reference september 2013 commander order british empire crystal palace director chairmen english football chairman investor,5
153,MattHancock,1978 alumni christ college cambridge alumni exeter college oxford british secretary common category conservative party english constituency privy council kingdom associate bank england educate king school chester chester dyslexia secretary health social care different 20102015 20152017 20172019 2019 british english november 2019 dmy date april 2020 ukparl indefinitely,5
196,RealDeniseWelch,1958 births actor county durham mountview academy theatre art hcards common category english autobiographer english soap opera actresses english television actress english television presenter english woman novelists participant british reality television series educate consett grammar school mood disorder reality show winner different british english october 2019 dmy date october 2019 woman equality party,5
205,SupremeDreams_1,1948 1st queen dragoon guard officer unsourced unsourced october 2020 british event rider british equestrian commander royal victorian order common category english olympic medallist equestrians 1972 summer olympics equestrian 1988 summer olympics graduate royal military academy sandhurst medalist 1972 summer olympics medalist 1988 summer olympics olympic equestrians great britain olympic gold medallist great britain olympic medalist equestrian olympic silver medallists great britain contain london gazette parameter supp set infobox military person embed educate marlborough college tetbury different sportspeople gloucestershire dmy date may 2014 webarchive bne,5
223,vivek_murthy,1977 write english men indian descent health profession kannada descent contain text cs1 cs1 maint unfit url common category harvard obama administration personnel huddersfield indian descent match surgeon general public health service commission corps officer english november 2020 mdy date february 2014 webarchive yale school management yale school medicine,5
228,cpeedell,english medical doctor english medical doctor cs1 maint archive copy title english politician national health action party,5
275,CraigJGill,1971 births 2016 death english drummer drummer suicide musician great manchester salford redirects connect item suicide england,5
279,kiranshaw,1953 births indian biologist indian businesspeople indian businesswoman indian woman scientist indian biologist indian businesspeople indian businesswoman indian woman scientist philanthropist write indian english unsourced hcards unsourced june 2020 businesspeople bangalore businesswoman karnataka cs1 maint numeric name author list common category federation australia female billionaire give pledger indian billionaire indian biotechnologists indian businesspeople pharmaceutical industry indian woman biologists mount carmel college bangalore citation bare urls citation title recipient padma bhushan science engineering recipient padma shri trade industry scientist bangalore match indian english september 2013 dmy date april 2017 winner nikkei asia prize,5
291,RevRichardColes,1962 births english anglican priest unsourced alumni king college london alumni college resurrection leeds hcards unsourced april 2019 bbc radio presenter bbc radio presenter english keyboardist english journalist english pop musician fellows king college london gay musician gay incomplete list november 202

In [102]:
# relabel and concatenate useful recursed clusters

label_df_ = label_df.append(uncategorized_df[uncategorized_df['label'].isin([0, 4])]).append(uncategorized_df_2[uncategorized_df_2['label'].isin([1, 3])]).append(uncategorized_df_3[uncategorized_df_3['label'].isin([3])])

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
# label_df.to_sql('labelstest', conn, if_exists='replace')

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
# label_df.to_excel('autolabel.xlsx')

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [103]:
label_df_.to_excel('curatedlabel.xlsx')

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
curated_label_df = pd.read_excel('curatedlabel.xlsx')
curated_label_df

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,screen_name,matched_cat,label
0,0,JoeBiden,1942 births 2008 candidate 2012 candidate lawyer politician politician write english contain potentially dated vague ambiguous time memoirists english descent french descent irish descent contain potentially date 2018 contain potentially dated september 2015 curlie biden family cs1 maint cs1 maint author parameter candidate 1988 presidential election candidate 2008 presidential election candidate 2020 presidential election catholic delaware catholic pennsylvania chairman senate committee foreign relation county council member commissioner delaware delaware democrat delaware fightin blue hens football player delaware lawyer democratic party presidential nominee democratic party vice presidential nominee democratic party senator democratic party vice president joe biden laetare medal recipients new castle county delaware politician obama administration cabinet member claymont delaware wilmington delaware involve plagiarism controversy politician scranton pennsylvania presidential medal freedom recipient public defender recipient recipient george order victory recipient order liberty ukraine recipient order cross terra mariana 1st class different syracuse college law alumni senator delaware delaware alumni pennsylvania faculty english february 2019 mdy date november 2020 vague ambiguous time october 2020 vice president widener faculty need clarification november 2020 need clarification october 2020 bibsys bnf gnd isni lccn nara nla nli nlp nta sudoc trove uscongress viaf worldcatid indefinitely,2
1,1,KamalaHarris,1964 births 2020 candidate politician woman politician politician woman politician woman activist san francisco bay area senator candidate president candidate vice president memoirists california politics woman politics woman lawyer write english dead external unsourced alpha kappa alpha baptist indian tamil descent politician indian descent politician jamaican descent prosecutor woman lawyer woman memoirists curlie links dead external march 2020 unsourced november 2020 biden administration cabinet member cs1 maint extra text author list california attorney general california democrat candidate 2020 presidential election democratic party senator district attorney california female senator female candidate president female candidate vice president harris family howard alumni joe biden 2020 presidential campaign kamala harris lawyer oakland california living member congress indian descent politician oakland california different senator california california hastings college law english july 2020 mdy date november 2020 bibsys gnd isni lccn uscongress viaf worldcatid woman california politics oakland california,2
2,2,ProjectLincoln,2019 establishment element joe biden 2020 presidential campaign never trump movement match political action committee mdy date june 2020 webarchive template wayback,3
3,3,BarackObama,1961 births protestant scholar politician president protestant scholar element activist illinois christian senator academic candidate president educator feminist hero icon legislator illinois write english nobel laureate protestant civil right lawyer community activist feminist gun control activists legal scholar memoirists english descent french descent german descent irish descent kenyan descent luo descent scottish descent swiss descent welsh descent political contain video clip curlie encyclopdia britannica internet archive librivox project gutenberg haudio microformats barack obama cs1 source candidate 2008 presidential election candidate 2012 presidential election columbia college new york democratic party presidential nominee democratic party president democratic party senator feature grammy award winner harvard law school illinois democrat illinois lawyer illinois senator lgbt right activist feminist member philosophical society nobel peace prize laureates nobelprize template property p8024 obama family official website different contain content include re

In [119]:
temp = curated_label_df[curated_label_df['label'] == 4]
print(len(temp))

815


C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
curated_label_df.to_sql('labels', conn, if_exists='replace')

C:\Users\epicp\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\epicp\Anaconda3\lib\site-packages\pandas\core\generic.py:2612: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,
